In [1]:
import requests
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, balanced_accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [2]:
# cols = ['Label', 'SYN Flag Count', 'Fwd Seg Size Min', 'FWD Init Win Bytes',
#        'FIN Flag Count', 'Average Packet Size', 'Packet Length Mean',
#        'Packet Length Max', 'Protocol', 'Idle Max', 'Idle Mean', 'Idle Min',
#        'Flow Duration', 'Fwd IAT Total', 'Fwd Packet Length Max',
#        'Fwd Segment Size Avg', 'Fwd Packet Length Mean',
#        'Bwd Packet Length Mean', 'Bwd Segment Size Avg', 'Packet Length Std',
#        'Bwd IAT Total', 'Bwd Packet Length Max']

cols = ['Label', 'Protocol', 'Flow Duration', 'Fwd Packet Length Max',
       'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd IAT Total',
       'Bwd IAT Total', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'FIN Flag Count', 'SYN Flag Count',
       'Down/Up Ratio', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Fwd Seg Size Min',
       'Idle Mean', 'Idle Max', 'Idle Min']

collection = [
    {"clf": AdaBoostClassifier(), "model": "AdaBoost"},
    {"clf": DecisionTreeClassifier(criterion="entropy"), "model": "DecisionTree"},
    {"clf": LinearSVC(), "model": "SVM"},
    {"clf": MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000), "model": "ANN"},
    {"clf": LogisticRegression(solver='newton-cg'), "model": "LogisticRegression"},
    {"clf": RandomForestClassifier(), "model": "RandomForest"},
    {"clf": XGBClassifier(use_label_encoder=False), "model": "XGBoost"},
    {"clf": BaggingClassifier(), "model": "Bagging"},
]

In [3]:
len(cols)

22

In [4]:
dataset = pd.read_csv("/media/kmdr7/Seagate/TA/DATASETS/newDatasetSampledEncoded.csv")[cols]

X = dataset.drop(columns=["Label"])
y = dataset["Label"]

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=1 / 7.0, random_state=1
)

In [30]:
dataset.to_numpy()[0]

array([1.00000000e+00, 6.00000000e+00, 1.97285100e+06, 1.49000000e+02,
       1.79518072e+00, 1.40887425e+03, 1.84142800e+06, 1.84094200e+06,
       1.44800000e+03, 7.05329341e+02, 7.22955645e+02, 1.00000000e+00,
       4.00000000e+00, 1.00000000e+00, 7.07447447e+02, 1.79518072e+00,
       1.40887425e+03, 2.92000000e+04, 3.20000000e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [32]:
np.transpose(scaler.fit_transform(dataset.to_numpy()[0][:, np.newaxis])).tolist()
# print(newds.max())
# print(newds.min())
# newds

[[-0.40000490065737887,
  -0.39999716905414845,
  2.6506537859660377,
  -0.3997760452017587,
  -0.40000367105300966,
  -0.3978278756352014,
  2.4474316876959668,
  2.4466801758619705,
  -0.3977673746824978,
  -0.3989157816552583,
  -0.3988885257384472,
  -0.40000490065737887,
  -0.4000002616954406,
  -0.40000490065737887,
  -0.3989125063840187,
  -0.40000367105300966,
  -0.3978278756352014,
  -0.3548538841124225,
  -0.3999569647173503,
  -0.40000644697802495,
  -0.40000644697802495,
  -0.40000644697802495]]

In [28]:
newds = scaler.fit_transform(dataset.to_numpy())
print(newds.max())
print(newds.min())

228.13263022232954
-2.290842827715566


In [ ]:
for col in collection:
    clf = col["clf"]
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = accuracy_score(y_test, y_pred)
    bacc = balanced_accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    try: roc_auc = roc_auc_score(y_test, y_pred)
    except: roc_auc = 0
    
    dump(clf, "/media/kmdr7/Seagate/TA/MODELS/ModelMinMax/" + col["model"] + ".joblib")
    
    requests.post(
        "http://localhost:8000/api/v1/training",
        json={
            "code": "005-WithMinMaxScaler",
            "algorithm": col["model"],
            "dataset": "IoT-23 Mirai 48-1 + IoT Traffic Traces",
            "matrix": {
                "tn": float(tn),
                "fp": float(fp),
                "fn": float(fn),
                "tp": float(tp)
            },
            "accuracy": float(acc),
            "balanced_accuracy": float(bacc),
            "recall": float(recall),
            "f1": float(f1),
            "precision": float(precision),
            "roc_auc": float(roc_auc),
        }
    )